In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance

In [7]:
##LECTURA DEL FICHERO SPSS Y SUS METADATOS
##Debe estar instalado el paquete de pyreadstat que permite leer ficheros SPSS y SAS
import pyreadstat

df, meta = pyreadstat.pyreadstat.read_sav('D:\TFM\Cuestionarios\CuestionariosEsp\Estudiantes\estudiantesESP1.sav', metadataonly=True)
df2, meta2 = pyreadstat.pyreadstat.read_sav('D:\TFM\Cuestionarios\CuestionariosEsp\Escuelas\Limpieza\escuelasESP1.sav')
df3, meta3 = pyreadstat.pyreadstat.read_sav('D:\TFM\Cuestionarios\CuestionariosEsp\profesores\Limpieza\profesoresESP1.sav', metadataonly=True)

In [8]:
pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 999
pd.options.display.max_columns = 9999

In [9]:
data = pd.read_csv('EstuTranTar.csv')

In [10]:
df2 = df2[['SC013Q01TA','CNTSCHID']]

In [19]:
data = pd.merge(data, df2,on='CNTSCHID')

In [35]:
#Eliminamos los valores nulos
data = data.dropna()

In [36]:
#Se borran las columnas que no van a entrar al modelo
## Tambien Eliminamos la altura y el peso, ya que da demasiada información
dataMod = data.drop(['Unnamed: 0','CNTSCHID','CNTSTUID','LECTURA1','MATES1','CIENCIA1','GLOBAL1','LECTURA2','MATES2','CIENCIA2','GLOBAL2','ST004D01T','WB151Q01HA','WB152Q01HA','ST003D02T','SC013Q01TA'],axis=1)

In [37]:
# training /test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
std_sc = StandardScaler()
x = std_sc.fit_transform(dataMod.values)
y = data['SC013Q01TA'].values

testsize = 0.2 # in the [0,1] range. 1: 100%
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=testsize, random_state=1)

In [38]:
from sklearn.ensemble import RandomForestClassifier

std_sc = StandardScaler()
x = std_sc.fit_transform(dataMod.values)
#x = dataMod.values



lr_m =  RidgeClassifier(alpha=0.134)

#ajuste sobre la muestra completa, sin split train-test
lr_m.fit(X_train, y_train)



RidgeClassifier(alpha=0.134)

# Validación del modelo con los datos de test

In [39]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_pred = np.clip(lr_m.predict(X_test), 0,  100.)



In [40]:
mat_confusion = confusion_matrix(
                    y_true    = y_test,
                    y_pred    = y_pred
                )

accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    =y_pred,
            normalize = True
           )

print("Matriz de confusión")
print("-------------------")
print(mat_confusion)
print("")
print(f"El accuracy de test es: {100 * accuracy} %")
    

Matriz de confusión
-------------------
[[3725  657]
 [1070 1531]]

El accuracy de test es: 75.26850923671773 %


In [41]:
#Dataframe con el coeficiente y nombre columna
VarCoef = pd.DataFrame(lr_m.coef_.T,dataMod.columns.to_list()).sort_values(0)
VarCoef ['Cols']=VarCoef.index

#Dataframe con las descripciones (convertimos un diccionario a un DataFrame)
Des1 = pd.DataFrame([[key, meta.column_names_to_labels[key]] for key in meta.column_names_to_labels.keys()], columns=['Cols', 'Desc'])
Des2 = pd.DataFrame([[key, meta2.column_names_to_labels[key]] for key in meta2.column_names_to_labels.keys()], columns=['Cols', 'Desc'])
Des3 = pd.DataFrame([[key, meta3.column_names_to_labels[key]] for key in meta3.column_names_to_labels.keys()], columns=['Cols', 'Desc'])
Des = pd.concat([Des1,Des2, Des3],axis=0)


#Hacemos todos los cruces, para que crucen todas las columnas
VarCoef0 = pd.merge(left=Des,right=VarCoef, left_on='Cols', right_on='Cols')

VarCoef0["Coefi_abs"] = abs(VarCoef0[0])
VarImp = VarCoef0.sort_values("Coefi_abs", ascending=False)
VarImp

,Cols,Desc,0,Coefi_abs
19,ST059Q01TA,Typically required to attend: Number of <class periods> per week in <test language lessons>,-0.078759,0.078759
30,LMINS,Learning time (minutes per week) - <test language>,0.051653,0.051653
29,MMINS,Learning time (minutes per week) - <Mathematics>,-0.046110,0.046110
20,ST059Q02TA,Typically required to attend: Number of <class periods> per week in mathematics,0.045712,0.045712
15,ST011D19TA,In your home: <Country-specific wealth item 3>,-0.030852,0.030852
4,ST011Q05TA,In your home: Educational software,0.022475,0.022475
12,ST011Q16NA,"In your home: Books on art, music, or design",-0.021656,0.021656
9,ST011Q10TA,In your home: Books to help with your school work,0.021392,0.021392
6,ST011Q07TA,In your home: Classic literature (e.g. <Shakespeare>),-0.020519,0.020519
7,ST011Q08TA,In your home: Books of poetry,0.020369,0.020369


In [42]:
# select alpha in ridge regression by CV
# First iteration to find alpha
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold

# define folds
n_folds = 10
kf = KFold(n_folds, shuffle=True)

# define param_grid
l_alpha = [2.**k for k in range(-6, 10)]
param_grid = {'alpha': l_alpha}

# define grid search
lr_m = RidgeClassifier()
ridge_alpha_search = GridSearchCV(lr_m, 
                                  param_grid=param_grid, 
                                  cv=kf, 
                                  scoring='neg_mean_absolute_error',
                                  n_jobs=2, 
                                  verbose=1)

_ = ridge_alpha_search.fit(x, y)

Fitting 10 folds for each of 16 candidates, totalling 160 fits


PicklingError: Could not pickle the task to send it to the workers.

In [ ]:
print("alpha_range: %.2f - %.2f" % (np.array(l_alpha).min(), np.array(l_alpha).max()))
print('alpha = %.2f' % (ridge_alpha_search.best_params_['alpha']))
print('best_cv_mae = %.3f' % (-ridge_alpha_search.best_score_))

plt.title("CV MAEs vs alpha")
plt.xticks(range(len(l_alpha)), l_alpha, rotation=45)
_ = plt.plot( -ridge_alpha_search.cv_results_['mean_test_score'])